In [ ]:
import numpy as np
import pandas as pd
from eofs.standard import Eof
import matplotlib.pyplot as plt

In [ ]:
def trans_linear_to_2D(data, side_length = 9):
    """Transforms an array of the shape (timesteps, number_grid_p) into (timesteps, sqrt(number_grid_p),sqrt(number_grid_p))"""
    t_length, linear_length = data.shape
    assert linear_length == side_length**2
    
    reshaped_data = np.reshape(data,(t_length,side_length,side_length), order = 'F')
    return reshaped_data

def trans_2D_into_linear(data):
    """Transforms an array of the shape (timesteps, sqrt(number_grid_p),sqrt(number_grid_p)) into (timesteps, number_grid_p)"""
    t_length, side_length, side_length_y= data.shape
    assert side_length == side_length_y
    reshaped_data = np.reshape(data,(t_length,side_length**2),order = 'F')
    
    return reshaped_data
    

In [ ]:
def reconstruct(eofs, pcs):
    """Function that reconstructs Fields from EOF and PC"""
    time_length, spatial_length = pcs.shape
    reconstructed_field = np.zeros(pcs.shape)
    
    for i in range(time_length):
        reconstructed_field[i,:] = np.sum(eofs*pcs[i,:,None], axis =0)
    return reconstructed_field

In [ ]:
def get_eofs(eof_rawdata):
    """Do EOF analysis and give out Eof, PC and corresponding variance"""
    solver = Eof(eof_rawdata,weights=None)
    eofs = solver.eofs()
    variance_fraction = solver.varianceFraction()
    pcs = solver.pcs()
    pseudo_pcs = solver.projectField(eof_rawdata)
    return eofs, pseudo_pcs, variance_fraction


In [ ]:
v_x_pandas=pd.read_csv('velocity_x.csv')
v_y_pandas=pd.read_csv('velocity_y.csv')

In [ ]:
v_x=v_x_pandas.values[:,1:]
v_y=v_y_pandas.values[:,1:]

**U and V Seperated analysed in 1D**

In [ ]:
eofs_x, pcs_x, variance_x = get_eofs(v_x)
eofs_y, pcs_y, variance_y = get_eofs(v_y)

**2D Analysis**, is not necessary, we demonstrate, that it does not matter if we do it in 2D or 1D

In [ ]:
v_x_2D = trans_linear_to_2D(v_x)
v_y_2D = trans_linear_to_2D(v_y)

In [ ]:
eofs_x_2D, pcs_x_2D, variance_x_2D = get_eofs(v_x_2D)

In [ ]:
plt.plot(variance_x_2D)
plt.plot(variance_x)

In [ ]:
trans_eofs_x_2D = trans_2D_into_linear(eofs_x_2D)

In [ ]:
plt.plot(eofs_x[0,:])
plt.plot(trans_eofs_x_2D[0,:])

**Test**, Can perfectly reconstruct field, and EoFs are orthogonal

In [ ]:
reconstructed_field_fun = reconstruct(eofs_x, pcs_x)
t_step = 100

In [ ]:
plt.plot(v_x[t_step,:],label = "act. field")
plt.plot(reconstructed_field_fun[t_step,:], label = "My rec")
plt.legend()

In [ ]:
#Test of Orthogonality
for j in range(81):
    for i in range(81):
        diag_sum = np.sum(eofs_x[j,:]*eofs_x[i,:])
        if diag_sum>1e-14:
            print(diag_sum, i, j)